# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [1]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from importlib import reload
from DataLoader import generar_rolling_features,crear_archivo_kaggle
import xgboost as xgb
import pandas as pd

In [2]:
ventas=importar_ventas()

In [3]:
ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'mean_discount', 'mean_diff_factory', 'std_discount', 'max_discount',
       'min_discount', 'std_diff_factory', 'max_diff_factory',
       'min_diff_factory', 'size', 'group', 'category', 'cluster', 'store_cod',
       'subgroup_cod', 'day', 'month', 'year'],
      dtype='object')

In [3]:
agregar_fourier(ventas,2)

In [4]:
ventas=generar_rolling_features(ventas,"mean_price",[1,2],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"demand",[1,2],["subgroup_cod","store_cod"])

In [6]:
#busquemos otra fecha para probar esto objetivo 01-01-2023 a 07-01-2023

# ventas_2023=ventas[ventas["date"]<"02-01-2023"]
# test_2023=ventas_2023.groupby(["subgroup_cod","store_cod"]).tail(7)
# train_2023=ventas_2023.drop(test_2023.index)

ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'mean_discount', 'mean_diff_factory', 'std_discount', 'max_discount',
       'min_discount', 'std_diff_factory', 'max_diff_factory',
       'min_diff_factory', 'size', 'group', 'category', 'cluster', 'store_cod',
       'subgroup_cod', 'day', 'month', 'year', 'anual_sin_1', 'anual_cos_1',
       'mensual_sin_1', 'mensual_cos_1', 'anual_sin_2', 'anual_cos_2',
       'mensual_sin_2', 'mensual_cos_2', 'mean_price_rolling_mean_7d',
       'mean_price_rolling_sum_7d', 'mean_price_rolling_mean_14d',
       'mean_price_rolling_sum_14d', 'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 'demand_rolling_mean_14d',
       'demand_rolling_sum_14d'],
      dtype='object')

In [4]:
import numpy as np
FEATURES=[
   #'mean_price', 
    'cluster', 
    #'store_cod', 
    #'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   'mensual_sin_1', 
    'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
   # 'mensual_sin_2', 
    #'mensual_cos_2',
   # "std_price",
    "max_price", 
   "min_price",
    #'mean_price_rolling_mean_7d',
   #    'mean_price_rolling_sum_7d',# 'mean_price_rolling_mean_14d',
       #'mean_price_rolling_sum_14d', 
   # 'mean_price_rolling_mean_21d',
     #  'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
    #  'mean_price_rolling_sum_28d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    'demand_rolling_mean_14d',
     #  'demand_rolling_sum_14d', #'demand_rolling_mean_21d',
       #'demand_rolling_sum_21d', #'demand_rolling_mean_28d',
      # 'demand_rolling_sum_28d'
    'std_discount', 'max_discount', 'min_discount', 'std_diff_factory',
    'max_diff_factory', 'min_diff_factory'
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso
test=preparar_test("ids_test.csv",ventas,FEATURES)
TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

#y_train_interno_log=np.log1p(y_train_interno)
#y_test_interno_log=np.log1p(y_test_interno)


# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

KeyError: "['demand_rolling_mean_14d'] not in index"

## Modelo interno

In [15]:
from sklearn.metrics import r2_score

reg = xgb.XGBRegressor(
    n_estimators=1000,
    eval_metric="rmse",
    eta=0.05,
    max_depth=6,
   #colsample_bytree=0.9, #reducir el numero de columnas aumenta la importancia de std_price
    early_stopping_rounds=50 
)

reg.fit(
    X_train_interno, y_train_interno
   ,eval_set=[(X_test_interno, y_test_interno)]   
   ,verbose=50                       
)
#sin anual_sin_2 450 estimadores =0.11283 y con da lo mismo

[0]	validation_0-rmse:3.17350
[50]	validation_0-rmse:2.16144
[100]	validation_0-rmse:2.04258
[150]	validation_0-rmse:1.93806
[200]	validation_0-rmse:1.86074
[250]	validation_0-rmse:1.81493
[300]	validation_0-rmse:1.78429
[350]	validation_0-rmse:1.76098
[400]	validation_0-rmse:1.74404
[450]	validation_0-rmse:1.72840
[500]	validation_0-rmse:1.71659
[550]	validation_0-rmse:1.70658
[600]	validation_0-rmse:1.69663
[650]	validation_0-rmse:1.68793
[700]	validation_0-rmse:1.67896
[750]	validation_0-rmse:1.67271
[800]	validation_0-rmse:1.66790
[850]	validation_0-rmse:1.66367
[900]	validation_0-rmse:1.65993
[950]	validation_0-rmse:1.65648
[999]	validation_0-rmse:1.65338


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=False, eta=0.05, eval_metric='rmse',
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None, ...)

In [21]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
std_diff_factory,0.234875
std_discount,0.124800
demand_rolling_mean_14d,0.109571
max_diff_factory,0.083618
min_diff_factory,0.075568
max_discount,0.060603
min_discount,0.054627
anual_sin_1,0.042850
year,0.038316
min_price,0.035366


## Modelo kraggel


In [13]:
from sklearn.metrics import mean_squared_error
import numpy as np

prediccion=reg.predict(X_test_interno)


# y_true_log y y_pred_log están en escala log1p
y_true_original = y_test_interno
y_pred_original = prediccion

rmse_original = np.sqrt(mean_squared_error(y_true_original, y_pred_original))

print(rmse_original)
print(r2_score(y_true_original,y_pred_original))

1.6420672803921805
0.7326104640960693


In [7]:
X_train=train[FEATURES]
y_train=train[TARGET]

reg = xgb.XGBRegressor(
    n_estimators=750,
    eval_metric="rmse",
    eta=0.05,
    max_depth=6,
   #colsample_bytree=0.9, #reducir el numero de columnas aumenta la importancia de std_price
    
)

reg.fit(
    X_train, y_train
)

prediccion=reg.predict(X_test_real)

kaggle=crear_archivo_kaggle(prediccion,test)

In [8]:
kaggle.isnull().sum()
# kaggle.loc[kaggle["TOTAL SALES"].isnull()]
# kaggle[['store_id', 'subgroup', 'date']] = kaggle["STORE_SUBGROUP_DATE_ID"].str.split('_', expand=True)
# nulls=kaggle.loc[kaggle["TOTAL SALES"].isnull()]

STORE_SUBGROUP_DATE_ID    0
TOTAL SALES               0
dtype: int64

In [78]:
nulls["subgroup"].unique()

array(['Basketball'], dtype=object)

In [29]:
test[['store_id', 'subgroup', 'date']] = test["store_subgroup_date_id"].str.split('_', expand=True)
test[test["subgroup"]=="Basketball"]






,date,store_subgroup_date_id,store_cod,subgroup_cod,mean_price,cluster,year,day,month,std_price,...,anual_sin_1,anual_cos_1,mensual_sin_1,mensual_cos_1,anual_sin_2,anual_cos_2,mensual_sin_2,mensual_cos_2,store_id,subgroup
33600,2024-01-01,S00001_Basketball_2024-01-01,0,32,NaN,0,2024,1,1,NaN,...,0.017202,0.999852,0.20495,0.978773,0.034398,0.999408,0.401198,0.915991,S00001,Basketball
33601,2024-01-02,S00001_Basketball_2024-01-02,0,32,NaN,0,2024,2,1,NaN,...,0.034398,0.999408,0.20495,0.978773,0.068755,0.997634,0.401198,0.915991,S00001,Basketball
33602,2024-01-03,S00001_Basketball_2024-01-03,0,32,NaN,0,2024,3,1,NaN,...,0.051584,0.998669,0.20495,0.978773,0.103031,0.994678,0.401198,0.915991,S00001,Basketball
33603,2024-01-04,S00001_Basketball_2024-01-04,0,32,NaN,0,2024,4,1,NaN,...,0.068755,0.997634,0.20495,0.978773,0.137185,0.990545,0.401198,0.915991,S00001,Basketball
33604,2024-01-05,S00001_Basketball_2024-01-05,0,32,NaN,0,2024,5,1,NaN,...,0.085906,0.996303,0.20495,0.978773,0.171177,0.985240,0.401198,0.915991,S00001,Basketball
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34645,2024-01-03,S00157_Basketball_2024-01-03,156,32,NaN,8,2024,3,1,NaN,...,0.051584,0.998669,0.20495,0.978773,0.103031,0.994678,0.401198,0.915991,S00157,Basketball
34646,2024-01-04,S00157_Basketball_2024-01-04,156,32,NaN,8,2024,4,1,NaN,...,0.068755,0.997634,0.20495,0.978773,0.137185,0.990545,0.401198,0.915991,S00157,Basketball
34647,2024-01-05,S00157_Basketball_2024-01-05,156,32,NaN,8,2024,5,1,NaN,...,0.085906,0.996303,0.20495,0.978773,0.171177,0.985240,0.401198,0.915991,S00157,Basketball
34648,2024-01-06,S00157_Basketball_2024-01-06,156,32,NaN,8,2024,6,1,NaN,...,0.103031,0.994678,0.20495,0.978773,0.204966,0.978769,0.401198,0.915991,S00157,Basketball


In [30]:
test.isnull().sum()

date                              0
store_subgroup_date_id            0
store_cod                         0
subgroup_cod                      0
mean_price                     1050
cluster                           0
year                              0
day                               0
month                             0
std_price                      1050
max_price                      1050
min_price                      1050
mean_price_rolling_mean_7d     1050
mean_price_rolling_sum_7d      1050
mean_price_rolling_mean_14d    1050
mean_price_rolling_sum_14d     1050
mean_price_rolling_mean_21d    1050
mean_price_rolling_sum_21d     1050
mean_price_rolling_mean_28d    1050
mean_price_rolling_sum_28d     1050
demand_rolling_mean_7d         1050
demand_rolling_sum_7d          1050
demand_rolling_mean_14d        1050
demand_rolling_sum_14d         1050
demand_rolling_mean_21d        1050
demand_rolling_sum_21d         1050
demand_rolling_mean_28d        1050
demand_rolling_sum_28d      

In [35]:
trans=leer_csv("eci_transactions")

trans[trans["subgroup"]=="Basketball"]

KeyboardInterrupt: 

In [67]:
test=preparar_test("ids_test.csv")

kaggle=DataLoader.crear_archivo_kaggle(prediccion,test)

In [68]:
kaggle.isnull().sum()

STORE_SUBGROUP_DATE_ID       0
TOTAL SALES               1050
dtype: int64

,DATE,STORE_SUBGROUP_DATE_ID,STORE_COD,SUBGROUP_COD,MEAN_PRICE,CLUSTER,YEAR,DAY,MONTH,STD_PRICE,...,DEMAND_ROLLING_SUM_21D,DEMAND_ROLLING_MEAN_28D,DEMAND_ROLLING_SUM_28D,STD_DISCOUNT,MAX_DISCOUNT,MIN_DISCOUNT,STD_DIFF_FACTORY,MAX_DIFF_FACTORY,MIN_DIFF_FACTORY,TOTAL SALES
0,2024-01-01,S00001_Laptops_2024-01-01,0,0,252.82,0,2024,1,1,0.0,...,94.000089,4.607139,128.999890,2.163747,9.18,4.59,14.820958,-31.44,-62.88,969.419743
1,2024-01-02,S00001_Laptops_2024-01-02,0,0,272.99,0,2024,2,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
2,2024-01-03,S00001_Laptops_2024-01-03,0,0,272.99,0,2024,3,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
3,2024-01-04,S00001_Laptops_2024-01-04,0,0,272.99,0,2024,4,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
4,2024-01-05,S00001_Laptops_2024-01-05,0,0,272.99,0,2024,5,1,0.0,...,96.000089,4.642853,129.999890,0.000000,-26.03,-26.03,0.000000,-28.36,-28.36,671.020470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77695,2024-01-03,S00157_Puzzles_2024-01-03,156,73,26.53,8,2024,3,1,0.0,...,104.001067,4.428645,124.002064,0.000000,0.17,0.17,0.000000,-8.07,-8.07,70.690697
77696,2024-01-04,S00157_Puzzles_2024-01-04,156,73,30.32,8,2024,4,1,0.0,...,105.001067,4.285765,120.001425,0.000000,3.31,3.31,0.000000,-9.40,-9.40,82.628069
77697,2024-01-05,S00157_Puzzles_2024-01-05,156,73,30.32,8,2024,5,1,0.0,...,105.001067,4.285765,120.001425,0.000000,3.31,3.31,0.000000,-9.40,-9.40,82.628069
77698,2024-01-06,S00157_Puzzles_2024-01-06,156,73,30.32,8,2024,6,1,0.0,...,105.001067,4.285765,120.001425,0.000000,3.31,3.31,0.000000,-9.40,-9.40,82.628069
